# Perfect basic

In [4]:
from prefect import flow

@flow
def my_favorite_function():
    print("What is your favorite function?")
    return 42

print(my_favorite_function())

/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/flows.py:243: UserWarning: A flow named 'my-favorite-function' and defined at '/tmp/ipykernel_10091/1517079621.py:3' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


21:03:49.522 | INFO    | prefect.engine - Created flow run 'amorphous-leopard' for flow 'my-favorite-function'

What is your favorite function?


21:03:49.702 | INFO    | Flow run 'amorphous-leopard' - Finished in state Completed()

42


## Run flows with parameters

In [5]:
import requests
from prefect import flow

@flow
def call_api(url):
    return requests.get(url).json()

api_result = call_api("http://time.jsontest.com/")
print(api_result)


21:07:24.144 | INFO    | prefect.engine - Created flow run 'honest-finch' for flow 'call-api'

21:07:24.938 | INFO    | Flow run 'honest-finch' - Finished in state Completed()

{'date': '05-20-2023', 'milliseconds_since_epoch': 1684591644723, 'time': '02:07:24 PM'}


## Run a basic flow with tasks

In [6]:
import requests
from prefect import flow, task

@task
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@flow
def api_flow(url):
    fact_json = call_api(url)
    return fact_json

print(api_flow("https://catfact.ninja/fact"))

21:08:28.441 | INFO    | prefect.engine - Created flow run 'interesting-dormouse' for flow 'api-flow'

21:08:28.521 | INFO    | Flow run 'interesting-dormouse' - Created task run 'call_api-0' for task 'call_api'

21:08:28.522 | INFO    | Flow run 'interesting-dormouse' - Executing 'call_api-0' immediately...

200


21:08:29.826 | INFO    | Task run 'call_api-0' - Finished in state Completed()

21:08:29.847 | INFO    | Flow run 'interesting-dormouse' - Finished in state Completed()

{'fact': 'There are more than 500 million domestic cats in the world, with approximately 40 recognized breeds.', 'length': 100}


### And of course we can create tasks that take input from and pass input to other tasks.

In [7]:
import requests
from prefect import flow, task

@task
def call_api(url):
    response = requests.get(url)
    print(response.status_code)
    return response.json()

@task
def parse_fact(response):
    fact = response["fact"]
    print(fact)
    return fact

@flow
def api_flow(url):
    fact_json = call_api(url)
    fact_text = parse_fact(fact_json)
    return fact_text

api_flow("https://catfact.ninja/fact")


/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'call_api' and defined at '/tmp/ipykernel_10091/1297208356.py:4' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/flows.py:243: UserWarning: A flow named 'api-flow' and defined at '/tmp/ipykernel_10091/1297208356.py:16' conflicts with another flow. Consider specifying a unique `name` parameter in the flow definition:

 `@flow(name='my_unique_name', ...)`
  warnings.warn(


21:09:30.975 | INFO    | prefect.engine - Created flow run 'bronze-corgi' for flow 'api-flow'

21:09:31.055 | INFO    | Flow run 'bronze-corgi' - Created task run 'call_api-0' for task 'call_api'

21:09:31.056 | INFO    | Flow run 'bronze-corgi' - Executing 'call_api-0' immediately...

200


...

21:09:32.345 | INFO    | Flow run 'bronze-corgi' - Executing 'parse_fact-0' immediately...

The domestic cat is the only species able to hold its tail vertically while walking. You can also learn about your cat's present state of mind by observing the posture of his tail.


21:09:32.399 | INFO    | Task run 'parse_fact-0' - Finished in state Completed()

21:09:32.418 | INFO    | Flow run 'bronze-corgi' - Finished in state Completed()

"The domestic cat is the only species able to hold its tail vertically while walking. You can also learn about your cat's present state of mind by observing the posture of his tail."

## Run a flow within a flow

In [8]:
from prefect import flow

@flow
def common_flow(config: dict):
    print("I am a subgraph that shows up in lots of places!")
    intermediate_result = 42
    return intermediate_result

@flow
def main_flow():
    # do some things
    # then call another flow function
    data = common_flow(config={})
    # do more things

main_flow()


21:12:14.493 | INFO    | prefect.engine - Created flow run 'warping-seahorse' for flow 'main-flow'

21:12:14.619 | INFO    | Flow run 'warping-seahorse' - Created subflow run 'blond-lemur' for flow 'common-flow'

I am a subgraph that shows up in lots of places!


21:12:14.679 | INFO    | Flow run 'blond-lemur' - Finished in state Completed()

21:12:14.698 | INFO    | Flow run 'warping-seahorse' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `int`'))]

## Parameter type conversion

In [9]:
from prefect import task, flow

@task
def printer(obj):
    print(f"Received a {type(obj)} with value {obj}")

# note that we define the flow with type hints
@flow
def validation_flow(x: int, y: str):
    printer(x)
    printer(y)

validation_flow(x="42", y=100)


22:01:31.429 | INFO    | prefect.engine - Created flow run 'attentive-bison' for flow 'validation-flow'

22:01:31.507 | INFO    | Flow run 'attentive-bison' - Created task run 'printer-0' for task 'printer'

22:01:31.509 | INFO    | Flow run 'attentive-bison' - Executing 'printer-0' immediately...

Received a <class 'int'> with value 42


22:01:31.569 | INFO    | Task run 'printer-0' - Finished in state Completed()

...

22:01:31.589 | INFO    | Flow run 'attentive-bison' - Executing 'printer-1' immediately...

Received a <class 'str'> with value 100


22:01:31.642 | INFO    | Task run 'printer-1' - Finished in state Completed()

22:01:31.663 | INFO    | Flow run 'attentive-bison' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]

In [10]:
from prefect import flow, task
from pydantic import BaseModel

class Model(BaseModel):
    a: int
    b: float
    c: str

@task
def printer(obj):
    print(f"Received a {type(obj)} with value {obj}")

@flow
def model_validator(model: Model):
    printer(model)

model_validator({"a": 42, "b": 0, "c": 55})


/home/dino/anaconda3/envs/torch/lib/python3.10/site-packages/prefect/tasks.py:298: UserWarning: A task named 'printer' and defined at '/tmp/ipykernel_10091/2818576951.py:9' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(


22:05:41.774 | INFO    | prefect.engine - Created flow run 'fluorescent-toad' for flow 'model-validator'

22:05:41.856 | INFO    | Flow run 'fluorescent-toad' - Created task run 'printer-0' for task 'printer'

22:05:41.857 | INFO    | Flow run 'fluorescent-toad' - Executing 'printer-0' immediately...

Received a <class '__main__.Model'> with value a=42 b=0.0 c='55'


22:05:41.914 | INFO    | Task run 'printer-0' - Finished in state Completed()

22:05:41.935 | INFO    | Flow run 'fluorescent-toad' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `NoneType`'))]